## Project prep
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
    - cohort pulled from the sql table 
    - sort all the endpoints 
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
    - divided web dev and data science (4 different programs)
    - sort all the endpoints 
    - counts of cohorts accessing same endpoints
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
    - active status indicator 
    - filter by the student being active 
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
    - figure out what constitutes a suspicious IP address
        - any ip addresses from china?
        - figure out general area of ip address
    - how to recognize web scraping
    - SKIP 
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
    - how to classify the endpoints as web dev and not this might be crazy
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
    - active status indicator 
    - each row of a ping would have student_active column - Parker 
    - 
7. Which lessons are least accessed?
    - kind of related to 2
8. Anything else I should be aware of?

1 is related to 7 and 2 

3 is relate to 6

anything extra we find

### Data Frame Must Haves
- log data
- cohort data
- Join on cohort id (Left Join) - Lupe has the join and the drop columns 
- column that has the active student at time of request (yes or no)
- need key for all the program ids (is_data_science maybe) 
- endpoint as a web dev, or data science 
    - Anything came before ada's start date is webdev


In [1]:
# prepare file for anomaly detection exercises
import pandas as pd
import numpy as np

# Vis tools
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans 
from datetime import datetime
# defining some functions to make it easier. will go in Wrangle function
from env import host, password, user
import os
import prepare, acquire

In [2]:
###################### Getting database Url ################
def get_db_url(db_name, user=user, host=host, password=password):
    """
        This helper function takes as default the user host and password from the env file.
        You must input the database name. It returns the appropriate URL to use in connecting to a database.
    """
    url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
    return url

######################### get generic data #########################
def get_any_data(database, sql_query):
    '''
    put in the query and the database and get the data you need in a dataframe
    '''

    return pd.read_sql(sql_query, get_db_url(database))

In [3]:
def acquire_curriculum_logs(csv_name = "anonymized-curriculum-access-07-2021.txt"):
    '''
    This function reads the csv of curriculum access logs
    If the file name needs to be changed change the default arguement
    '''
    # assign column names to use
    colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
    # read csv
    df = pd.read_csv(csv_name, 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 2, 3, 4, 5])
    return df

def make_datetime_index(df, col_name):
    '''
    This function takes in a dataframe 
    A column name of the column that is your date (as string)
    Performs basic to_datetime conversion and sets tha column as the index
    '''
    
    df[col_name] = pd.to_datetime(df[col_name])

    df = df.set_index(col_name)
    
    return df

def prep(df):
    '''
    This function takes in the curriculum log data, and the user number you want to look at
    returns a dataframe of the pages they accessed and when
    '''
    #df = df[df.user_id == user]
    df = make_datetime_index(df, 'date')
    pages = df['endpoint'].resample('d').count()
    return pages

In [4]:
df = pd.read_csv('anonymized-curriculum-access.csv')

In [5]:
dbase = 'curriculum_logs'
sqlquery = 'SELECT * FROM cohorts'

cohorts = get_any_data(dbase, sqlquery)

In [6]:
log_df = df.merge(cohorts, how = 'left', left_on = 'cohort_id', right_on = 'id')

In [7]:
log_df.sample(50)

,Unnamed: 0,date,endpoint,user_id,cohort_id,source_ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
990916,990916,2021-06-22,java-ii,935,138.0,99.43.138.137,138.0,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,None,2.0
287845,287845,2019-06-24,toc,211,28.0,107.77.169.3,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
182989,182989,2019-01-24,java-ii/annotations,292,27.0,97.105.19.58,27.0,Yosemite,#yosemite,2018-11-05,2019-04-03,2018-11-05 15:26:37,2018-11-05 15:26:37,None,2.0
328306,328306,2019-08-21,1-fundamentals/1.1-intro-to-data-science,481,34.0,97.105.19.58,34.0,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,None,3.0
235145,235145,2019-04-03,javascript-i/functions,393,31.0,97.105.19.58,31.0,Andromeda,#andromeda,2019-03-18,2019-07-30,2019-03-18 20:35:06,2019-03-18 20:35:06,None,2.0
98554,98554,2018-08-06,java-i,200,24.0,97.105.19.58,24.0,Voyageurs,#voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,None,2.0
955614,955614,2021-06-01,java-iii/mvc/accessing-data,872,135.0,72.177.147.207,135.0,Marco,#marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,None,2.0
127600,127600,2018-09-20,html-css/css-i/introduction,270,26.0,97.105.19.58,26.0,Xanadu,#xanadu,2018-09-17,2019-02-08,2018-09-17 19:09:51,2018-09-17 19:09:51,None,2.0
154098,154098,2018-11-09,html-css/css-i/positioning,306,27.0,97.105.19.58,27.0,Yosemite,#yosemite,2018-11-05,2019-04-03,2018-11-05 15:26:37,2018-11-05 15:26:37,None,2.0
80700,80700,2018-07-01,jquery/events/mouse-events,191,24.0,108.199.115.75,24.0,Voyageurs,#voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,None,2.0


In [8]:
log_df = log_df.drop(columns = ['Unnamed: 0','slack','created_at','updated_at','deleted_at'])

In [9]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018810 entries, 0 to 1018809
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   date        1018810 non-null  object 
 1   endpoint    1018809 non-null  object 
 2   user_id     1018810 non-null  int64  
 3   cohort_id   965313 non-null   float64
 4   source_ip   1018810 non-null  object 
 5   id          954313 non-null   float64
 6   name        954313 non-null   object 
 7   start_date  954313 non-null   object 
 8   end_date    954313 non-null   object 
 9   program_id  954313 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 85.5+ MB


In [10]:
log_df = make_datetime_index(log_df,'date' )

In [11]:
log_df.isnull().value_counts()

endpoint  user_id  cohort_id  source_ip  id     name   start_date  end_date  program_id
False     False    False      False      False  False  False       False     False         954312
                   True       False      True   True   True        True      True           53497
                   False      False      True   True   True        True      True           11000
True      False    False      False      False  False  False       False     False              1
dtype: int64

In [12]:
log_df = log_df.dropna()

In [13]:
log_df.shape

(954312, 9)

In [14]:
log_df.start_date = pd.to_datetime(log_df.start_date.copy())

In [15]:
log_df.end_date = pd.to_datetime(log_df.end_date.copy())

In [16]:
log_df['is_active'] = (log_df.index >= log_df.start_date) & (log_df.index <= log_df.end_date)

In [17]:
log_df.sample(50)

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2021-05-08,java-i,760,62.0,72.190.237.130,62.0,Jupiter,2020-09-21,2021-03-30,2.0,False
2020-10-06,mysql/databases,778,51.0,66.69.41.190,51.0,Deimos,2019-09-16,2020-02-27,2.0,False
2020-04-14,mysql/basic-statements,428,28.0,70.121.183.95,28.0,Staff,2014-02-04,2014-02-04,2.0,False
2018-06-21,spring/fundamentals/services,127,23.0,97.105.19.61,23.0,Ulysses,2018-03-05,2018-07-19,2.0,True
2018-10-19,appendix/further-reading/javascript/array-splice,270,26.0,97.105.19.58,26.0,Xanadu,2018-09-17,2019-02-08,2.0,True
2020-08-26,html-css,494,51.0,66.25.154.64,51.0,Deimos,2019-09-16,2020-02-27,2.0,False
2021-04-12,html-css,816,28.0,76.187.230.164,28.0,Staff,2014-02-04,2014-02-04,2.0,False
2020-05-15,5-stats/4.4-compare-group-membership,581,55.0,72.190.187.173,55.0,Curie,2020-02-03,2020-07-07,3.0,True
2019-07-19,javascript-ii/modules,418,32.0,97.105.19.58,32.0,Betelgeuse,2019-05-28,2019-10-08,2.0,True


In [18]:
active_df = log_df[log_df.is_active == True]

In [19]:
active_df.groupby(by = 'user_id').sum()

,cohort_id,id,program_id,is_active
user_id,,,,
2,33902.0,33902.0,3082.0,1541
3,33308.0,33308.0,3028.0,1514
4,15224.0,15224.0,1384.0,692
5,35662.0,35662.0,3242.0,1621
6,28886.0,28886.0,2626.0,1313
...,...,...,...,...
991,122181.0,122181.0,1758.0,879
993,47260.0,47260.0,680.0,340
994,26966.0,26966.0,388.0,194


In [20]:
inactive_df = log_df[log_df.is_active == False]

In [21]:
(inactive_df[inactive_df['program_id'] == 3]).endpoint.value_counts().head(30)

/                                                                1782
search/search_index.json                                          649
classification/overview                                           368
sql/mysql-overview                                                343
classification/scale_features_or_not.svg                          325
anomaly-detection/AnomalyDetectionCartoon.jpeg                    226
anomaly-detection/overview                                        225
fundamentals/AI-ML-DL-timeline.jpg                                223
fundamentals/modern-data-scientist.jpg                            221
fundamentals/intro-to-data-science                                217
classification/classical_programming_vs_machine_learning.jpeg     187
1-fundamentals/modern-data-scientist.jpg                          130
1-fundamentals/AI-ML-DL-timeline.jpg                              129
1-fundamentals/1.1-intro-to-data-science                          129
classification/prep 

In [22]:
def active_status(df, start, end):
    df['is_active'] = (df.index >= df[start]) & (df.index <= df[end])
    return df

In [23]:
active_status(log_df, 'start_date', 'end_date')

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0,True
...,...,...,...,...,...,...,...,...,...,...
2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0,True


In [24]:
def make_datetime(df, col_name, set_index = False):
    '''
    This function takes in a dataframe 
    A column name of the column that is your date (as string)
    Performs basic to_datetime conversion and sets tha column as the index
    '''
    if set_index == True:
        df[col_name] = pd.to_datetime(df[col_name])
        df = df.set_index(col_name)
        return df
    else:
        df[col_name] = pd.to_datetime(df[col_name])
    return df

In [25]:
make_datetime(log_df, 'start_date')

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0,True
...,...,...,...,...,...,...,...,...,...,...
2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0,True


In [26]:
active_df['pings'] = 1

<ipython-input-26-d9354e190a6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_df['pings'] = 1


In [27]:
active_df.groupby(by = 'user_id').sum()

,cohort_id,id,program_id,is_active,pings
user_id,,,,,
2,33902.0,33902.0,3082.0,1541,1541
3,33308.0,33308.0,3028.0,1514,1514
4,15224.0,15224.0,1384.0,692,692
5,35662.0,35662.0,3242.0,1621,1621
6,28886.0,28886.0,2626.0,1313,1313
...,...,...,...,...,...
991,122181.0,122181.0,1758.0,879,879
993,47260.0,47260.0,680.0,340,340
994,26966.0,26966.0,388.0,194,194


In [28]:
active_users = active_df.groupby(by = 'user_id').sum()

In [29]:
active_users.describe().T

,count,mean,std,min,25%,50%,75%,max
cohort_id,741.0,60486.836707,60484.006240,57.0,21450.0,41856.0,78088.0,462434.0
id,741.0,60486.836707,60484.006240,57.0,21450.0,41856.0,78088.0,462434.0
program_id,741.0,2082.519568,1412.083426,2.0,1140.0,1899.0,2826.0,9672.0
is_active,741.0,968.415655,638.915976,1.0,536.0,901.0,1311.0,4786.0
pings,741.0,968.415655,638.915976,1.0,536.0,901.0,1311.0,4786.0


In [30]:
user_pings = pd.DataFrame(active_users['pings'])

In [31]:
user_pings

,pings
user_id,
2,1541
3,1514
4,692
5,1621
6,1313
...,...
991,879
993,340
994,194


In [32]:
log_df.describe()

,user_id,cohort_id,id,program_id
count,954312.000000,954312.000000,954312.000000,954312.000000
mean,498.817419,54.436431,54.436431,2.084774
std,274.303663,39.285234,39.285234,0.404666
min,1.000000,1.000000,1.000000,1.000000
25%,282.000000,28.000000,28.000000,2.000000
50%,509.000000,34.000000,34.000000,2.000000
75%,705.000000,59.000000,59.000000,2.000000
max,1082.000000,139.000000,139.000000,4.000000


In [33]:
cohorts = pd.read_csv('full_cohort_list.csv')

In [34]:
cohorts

,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
5,6,Franklin,#franklin,2015-02-03,2015-05-26,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
6,7,Glacier,#glacier,2015-06-05,2015-10-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
7,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
8,9,Apollo,#apollo,2015-03-30,2015-07-29,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,4
9,10,Balboa,#balboa,2015-11-03,2016-03-11,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,4


In [35]:
log_df = df.merge(cohorts, how = 'left', left_on = 'cohort_id', right_on = 'id')

In [36]:
log_df.value_counts().sum()

11000

In [37]:
log_df

,Unnamed: 0,date,endpoint,user_id,cohort_id,source_ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,0,2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,1,2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018805,1018805,2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,NaN,2.0
1018806,1018806,2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,NaN,2.0
1018807,1018807,2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,NaN,2.0
1018808,1018808,2021-07-15,java-ii/object-oriented-programming,933,138.0,72.190.28.51,138.0,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,NaN,2.0


In [38]:
log_df = log_df.drop(columns = ['Unnamed: 0','slack','created_at','updated_at','deleted_at'])

In [39]:
log_df

,date,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id
0,2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...,...
1018805,2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0
1018806,2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0
1018807,2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0
1018808,2021-07-15,java-ii/object-oriented-programming,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0


In [40]:
log_df = make_datetime_index(log_df,'date' )

In [41]:
log_df

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id
date,,,,,,,,,
2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...
2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0
2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0
2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0


In [42]:
for x in log_df[['start_date','end_date']]:
    log_df = make_datetime(log_df, x)

In [43]:
prepare.active_status(log_df, log_df.index, 'start_date', 'end_date')

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id,is_active
date,,,,,,,,,,
2018-01-26,/,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0,False
2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0,True
...,...,...,...,...,...,...,...,...,...,...
2021-07-15,toc,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,138.0,Neptune,2021-03-15,2021-09-03,2.0,True
2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,138.0,Neptune,2021-03-15,2021-09-03,2.0,True


In [44]:
inactive_df = log_df[log_df.is_active == False]

In [45]:
active_df = log_df[log_df.is_active == True]

In [46]:
active_ping = active_df.groupby(by = 'user_id').sum()

In [47]:
active_ping.sort_values(by = 'is_active').head(50)

,cohort_id,id,program_id,is_active
user_id,,,,
918,138.0,138.0,2.0,1
879,135.0,135.0,2.0,1
619,57.0,57.0,2.0,1
940,138.0,138.0,2.0,1
832,186.0,186.0,6.0,3
278,96.0,96.0,8.0,4
539,260.0,260.0,10.0,5
956,834.0,834.0,12.0,6
812,406.0,406.0,14.0,7


In [48]:
active_ping

,cohort_id,id,program_id,is_active
user_id,,,,
2,33902.0,33902.0,3082.0,1541
3,33308.0,33308.0,3028.0,1514
4,15224.0,15224.0,1384.0,692
5,35662.0,35662.0,3242.0,1621
6,28886.0,28886.0,2626.0,1313
...,...,...,...,...
1076,15438.0,15438.0,186.0,93
1078,7304.0,7304.0,88.0,44
1079,9960.0,9960.0,120.0,60


# acquire use

In [49]:
log_df = acquire.get_joined_curriculum_data()

/Users/parkervoit/Desktop/codeup-data-science/anomaly-detection-project/acquire.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(csv_name,
